## APRENDIZAJE PROFUNDO PARA LA DETECCIÓN DE CAÍDAS EN PERSONAS VULNERABLES
## PRUEBA FINAL - MARÍA PAULA SAA BELTRÁN

In [7]:
import cv2                              # Librería de visión artificial
import numpy as np                      # Librería para cálculo numérico
import os                               # Módulo para trabajar con so
from matplotlib import pyplot as plt    # Librería de visualización de datos
import time                             # Función de temporizador
import mediapipe as mp                  # Modelos de ML para detección

mp_holistic = mp.solutions.holistic        # Modelo Holistic
mp_drawing = mp.solutions.drawing_utils    # Utilidades de dibujo para los puntos de referencia

def mediapipe_detection(image, model):             
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convertir imagen de BGR a RGB
    image.flags.writeable = False                  # Imagen no apta para uso
    results = model.process(image)                 # Predicción
    image.flags.writeable = True                   # Imagen apta para uso
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Convertir imagen de RGB a BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)       # Conexiones de todo el cuerpo

def draw_styled_landmarks(image, results): # Hacer las línes con colores determinados
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,                    # Dibujar conexiones de todo el cuerpo
                              mp_drawing.DrawingSpec(color=(51, 40, 28), thickness = 4, circle_radius = 4),
                              mp_drawing.DrawingSpec(color=(176, 201, 72), thickness = 4, circle_radius = 2)) 

def extract_keypoints(results):
    # Son 3 valores (x, y, z) y 4 para pose (visivility) - res.x, res.y, res.z, res.visibility -> Cada uno es un valor individual de cada punto de referencia
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # Concatenar todos los puntos
    return np.concatenate([pose])

colors = [(0, 0, 97), (255, 0, 0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
    
    return output_frame

actions=['NO CAIDA', 'CAIDA']

from keras.models import load_model
model = load_model('modelo_tesis_mpsaab_final_201122_mami.h5')


In [8]:

# New detection variables
sequence = [] # concatenate a data on a sequence
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(2)
# Access an holistic model - mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # Make detections 
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        # Draw Landmawks
        draw_styled_landmarks(image, results)
        # 2. Prediction logic 
        keypoints = extract_keypoints(results)
        #sequence.insert(0, keypoints)
        sequence.append(keypoints)
        sequence = sequence[-50:]

        if len(sequence) == 50:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

        # 3. Visualization logic
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]: # current actions was not equal to the last
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
                if len(sentence) > 5:
                    sentence = sentence[-5:]
            
            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (169, 0, 227), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (231, 0, 60), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow("OpenCV Feed", image)
        # Break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


QObject::moveToThread: Current thread (0x564ac1885300) is not the object's thread (0x564ac31eecd0).
Cannot move to target thread (0x564ac1885300)

QObject::moveToThread: Current thread (0x564ac1885300) is not the object's thread (0x564ac31eecd0).
Cannot move to target thread (0x564ac1885300)

QObject::moveToThread: Current thread (0x564ac1885300) is not the object's thread (0x564ac31eecd0).
Cannot move to target thread (0x564ac1885300)

QObject::moveToThread: Current thread (0x564ac1885300) is not the object's thread (0x564ac31eecd0).
Cannot move to target thread (0x564ac1885300)

QObject::moveToThread: Current thread (0x564ac1885300) is not the object's thread (0x564ac31eecd0).
Cannot move to target thread (0x564ac1885300)

QObject::moveToThread: Current thread (0x564ac1885300) is not the object's thread (0x564ac31eecd0).
Cannot move to target thread (0x564ac1885300)

QObject::moveToThread: Current thread (0x564ac1885300) is not the object's thread (0x564ac31eecd0).
Cannot move to tar

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti